In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time

# Import API key
from api_keys import API_KEY

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "./output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

In [3]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []
country_codes = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1])
    city_name = city.city_name
    country_code = city.country_code
    #print(f"City is {city_name} & its country code is {country_code}")
    
    # If the city is unique, then add it to a our cities list & also add its corresponding country code
    if city_name not in cities:
        cities.append(city_name)
        country_codes.append(country_code)

# Print the city count to confirm sufficient count
print('The above script has generated {} different cities using random numbers.'.format(len(set(cities))))

The above script has generated 622 different cities using random numbers.


In [6]:
#create a dataframe to store all the cities and countries generated with citipy
df_countries = pd.DataFrame({'Cities': cities, 'Country': country_codes})

#add additional blank columns to store information from openweathermap api
df_countries['Latitude'] = ''
df_countries['Longitude'] = ''
df_countries['Temperature (F)'] = ''
df_countries['Humidity (%)'] = ''
df_countries['Cloudiness (%)'] = ''
df_countries['Wind Speed (mph)'] = ''

# keeps count of those cities whose weather information is successfully obtained
city_count = 0

#loop through all rows and fill in values for blank columns in dataframe
for index,row in df_countries.iterrows():
    city_name = row['Cities']
    country_id = row['Country']
    
    #api call for all Cities 
    api_url = "http://api.openweathermap.org/data/2.5/weather" \
    "?q={},{}&units=IMPERIAL&mode=json&APPID={}".format(city_name,country_id,API_KEY)
    
    country_info = requests.get(api_url).json()
    
    try:
        df_countries.loc[index,'Latitude'] = country_info['coord']['lat']
        df_countries.loc[index,'Longitude'] = country_info['coord']['lon']
        df_countries.loc[index,'Temperature (F)'] = country_info['main']['temp']
        df_countries.loc[index,'Humidity (%)'] = country_info['main']['humidity']
        df_countries.loc[index,'Cloudiness (%)'] = country_info['clouds']['all']
        df_countries.loc[index,'Wind Speed (mph)'] = country_info['wind']['speed']
        
        #print log for each city
        print(f"Processing Record for {index} | city named: {city_name}")
        # only increment if you get all the weather information
        city_count = city_count + 1
    
    except KeyError:
        df_countries.loc[index,'Latitude'] = np.nan
        df_countries.loc[index,'Longitude'] = np.nan
        df_countries.loc[index,'Temperature (F)'] = np.nan
        df_countries.loc[index,'Humidity (%)'] = np.nan
        df_countries.loc[index,'Cloudiness (%)'] = np.nan
        df_countries.loc[index,'Wind Speed (mph)'] = np.nan
        print(f"Missing weather information for city named: {city_name} ...hence skipping")
        
print(f"Processing successfully completed. Retrieved weather information for {city_count} cities")


Missing weather information for city named: barentsburg ...hence skipping
Processing Record for 1 | city named: bajil
Processing Record for 2 | city named: baracoa
Processing Record for 3 | city named: karabanovo
Processing Record for 4 | city named: maldonado
Processing Record for 5 | city named: carnarvon
Processing Record for 6 | city named: yeniseysk
Processing Record for 7 | city named: rikitea
Processing Record for 8 | city named: labuan
Missing weather information for city named: marzuq ...hence skipping
Processing Record for 10 | city named: richards bay
Processing Record for 11 | city named: albany
Processing Record for 12 | city named: poole
Processing Record for 13 | city named: anqing
Processing Record for 14 | city named: buala
Processing Record for 15 | city named: ugljevik
Missing weather information for city named: odweyne ...hence skipping
Processing Record for 17 | city named: hithadhoo
Processing Record for 18 | city named: jamestown
Missing weather information for c

In [17]:
#change all data recieved from openweathermap api to numerical data
df_countries['Latitude'] = pd.to_numeric(df_countries['Latitude'])
df_countries['Longitude'] = pd.to_numeric(df_countries['Longitude'])
df_countries['Temperature (F)'] = pd.to_numeric(df_countries['Temperature (F)'])
df_countries['Humidity (%)'] = pd.to_numeric(df_countries['Humidity (%)'])
df_countries['Cloudiness (%)'] = pd.to_numeric(df_countries['Cloudiness (%)'])
df_countries['Wind Speed (mph)'] = pd.to_numeric(df_countries['Wind Speed (mph)'])

#display dataframe df_countries after openweathermap api calls removing those with NaNs elements
df_countries = df_countries.dropna()
print(df_countries.count())
print(df_countries.head(20))
df_countries.dtypes

Cities              540
Country             540
Latitude            540
Longitude           540
Temperature (F)     540
Humidity (%)        540
Cloudiness (%)      540
Wind Speed (mph)    540
dtype: int64
          Cities Country  Latitude  Longitude  Temperature (F)  Humidity (%)  \
1          bajil      ye     15.06      43.29            63.56          86.0   
2        baracoa      cu     20.35     -74.50            76.43         100.0   
3     karabanovo      ru     56.31      38.70            21.44          93.0   
4      maldonado      uy    -34.91     -54.96            68.00          72.0   
5      carnarvon      au    -24.87     113.63            75.20          53.0   
6      yeniseysk      ru     58.45      92.17           -14.30          65.0   
7        rikitea      pf    -23.12    -134.97            76.56         100.0   
8         labuan      my      5.33     115.20            79.70         100.0   
10  richards bay      za    -28.77      32.06            75.48          81.

Cities               object
Country              object
Latitude            float64
Longitude           float64
Temperature (F)     float64
Humidity (%)        float64
Cloudiness (%)      float64
Wind Speed (mph)    float64
dtype: object